In [ ]:
import asyncio
import planet
from planet import Auth, Session, data_filter
import getpass
import geopandas as gpd
import pandas as pd
from shapely.geometry import box
import json
import datetime

In [ ]:
user = input("Username: ")
pw = getpass.getpass("Password: ")
auth = Auth.from_login(user,pw)
auth.store()

In [ ]:
class SearchFeatures:
    """Class to prepare features for Planet api search and order"""
    def __init__(self, features_fp, bounding_box=False):
        """Initialize the SearchFeatures object."""
        self.features_fp = features_fp
        self.bounding_box = bounding_box
        self.gdf = gpd.read_file(features_fp)
        self.json_data = None
    
    def convert_to_wgs(self):
        """Convert the GeoDataFrame CRS to WGS84 (EPSG:4326)."""
        self.gdf = self.gdf.to_crs(epsg=4326)

    def geom_to_bbox(self):
        """Convert geometries to bounding boxes if specified."""
        if self.bounding_box:
            self.gdf["geometry"] = self.gdf.geometry.apply(lambda geom: box(*geom.bounds))

    def features_to_json(self):
        """Convert features to GeoJSON format."""
        json_text = self.gdf.to_json()
        self.json_data = json.loads(json_text)

    def filter_json(self, id_col, id_val):
        """Filter the GeoJSON features based on a given column and value."""
        filtered = [d for d in self.json_data["features"] if d["properties"].get(id_col) == id_val]
        return filtered[0] if filtered else None

    def process(self):
        """Run all necessary processing and return GeoJSON."""
        self.convert_to_wgs()
        self.geom_to_bbox()
        self.features_to_json()
        return self.json_data
        

In [ ]:
search_features = SearchFeatures("inputs/test_shape.shp", bounding_box=True)
json_data = search_features.process()
filtered_feature = search_features.filter_json("id", 1)

In [ ]:
def create_img_filter(start_date, end_date, geometry_filter, max_cloud_percent=10):
    start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    return data_filter.and_filter([
        data_filter.permission_filter(),
        data_filter.date_range_filter("acquired", gte=start, lte=end),
        data_filter.geometry_filter(geometry_filter),
        data_filter.range_filter("cloud_percent", lte=max_cloud_percent, gte=0)
    ])

In [ ]:
def available_img_to_df(response_lst):
    data = []
    for item in response_lst:
        data.append({
            "id": item["id"],
            "cloud_percent": item["properties"]["cloud_percent"],
            "satellite_id": item["properties"]["satellite_id"]
        })
    
    return pd.DataFrame(data)

In [ ]:
async def get_available(input_filter):
    async with Session() as sess:
        cl = sess.client('data')
        return [i async for i in cl.search(['PSScene'], input_filter)]


In [ ]:
search_filter = create_img_filter("2024-04-01", "2024-05-30", filtered_feature)
available_images = await get_available(search_filter)
available_img_to_df(available_images)

### Get thumbnail....?

In [ ]:
def create_request(geom, item_ids, order_ref):
   new_order = planet.order_request.build_request(
       name=order_ref,
       products=[
           planet.order_request.product(item_ids=item_ids,
                                        product_bundle='analytic_udm2',
                                        item_type='PSScene')
       ],
       tools=[planet.order_request.clip_tool(aoi=geom)])

   return new_order

In [ ]:
request = create_request(test_geom, ["20240518_103839_43_24bc"], "sg_test_order")

In [ ]:
async def main(arequest):
    # Create a session and client
    # The Orders API client is also a subclass of the Session
    # class, so it has all the same methods.
    async with planet.Session() as sess:

        # 'orders' is the service name for the Orders API.
        cl = sess.client("orders")
        order = await cl.create_order(arequest)
        return order

new_order = await main(request)

In [ ]:
from planet import reporting

async def create_wait_and_download():
    async with Session() as sess:
        cl = sess.client('orders')
        with reporting.StateBar(state='creating') as bar:
            # create order
            order = await cl.create_order(request)
            bar.update(state='created', order_id=order['id'])

            # poll
            await cl.wait(order['id'], callback=bar.update_state)

        # download
        await cl.download_order(order['id'])

create_wait_and_download()

In [ ]:
async def create_and_download(client, order_detail, directory):
   with planet.reporting.StateBar(state='creating') as reporter:
       order = await client.create_order(order_detail)
       reporter.update(state='created', order_id=order['id'])
       await client.wait(order['id'], callback=reporter.update_state)

   await client.download_order(order['id'], directory, progress_bar=True)

async def main():
   async with planet.Session() as sess:
       cl = sess.client('orders')

       # Create the order request
       request = create_request()

       # Create and download the order
       order = await create_and_download(cl, request, DOWNLOAD_DIR)
